# Examples

In [ ]:
%load_ext autoreload
%autoreload 2

### GeoNames City Dataset

In [ ]:
import pandas as pd
geonames = pd.read_parquet('https://paper.jupyter-scatter.dev/geonames.pq')
geonames.head(2)

In [ ]:
import jscatter

scatter = jscatter.Scatter(
    data=geonames,
    x='Longitude',
    y='Latitude',
    color_by='Continent',
    width=600,
    height=600
)
scatter.show()

In [ ]:
import matplotlib

scatter.opacity(0.5)
scatter.size(by='Population', map=(1, 8, 10), norm=matplotlib.colors.AsinhNorm())
scatter.color(by='Population', map='magma', norm=matplotlib.colors.LogNorm(), order='reverse')

In [ ]:
scatter.legend(True)
scatter.axes(True, labels=True)
scatter.tooltip(True, properties=['color', 'Latitude', 'Country'], preview='Name')

In [ ]:
scatter.xy(x='Mercator X', y='Mercator Y')

In [ ]:
scatter.selection(geonames.query('Population > 10_000_000').index)

In [ ]:
geonames.iloc[scatter.selection()]

### Fashion MNIST Embeddings

In [ ]:
import pandas as pd
fashion_mnist = pd.read_parquet('https://paper.jupyter-scatter.dev/fashion-mnist-embeddings.pq')

In [ ]:
import anywidget
import traitlets
import traittypes

class ImagesWidget(anywidget.AnyWidget):
    _esm = """
    const baseUrl = 'https://paper.jupyter-scatter.dev/fashion-mnist-images/';
    export function render({ model, el }) {
      const container = document.createElement('div');
      container.classList.add('images-container');
      el.appendChild(container);

      const grid = document.createElement('div');
      grid.classList.add('images-grid');
      container.appendChild(grid);

      function renderImages() {
        grid.textContent = '';
        
        model.get('images').forEach((image) => {
          const imgId = String(image).padStart(5, '0');
        
          const img = document.createElement('div');
          img.classList.add('images-fashion-mnist');
          img.style.backgroundImage = `url(${baseUrl}${imgId}.png)`;
        
          grid.appendChild(img);
        });
      }

      model.on("change:images", renderImages);
      
      renderImages();
    }
    """

    _css = """
    .images-container {
      position: absolute;
      inset: 0;
      overflow: auto;
      background: black;
    }
    
    .images-grid {
      display: grid;
      grid-template-columns: repeat(auto-fit, minmax(32px, 1fr));
      align-content: flex-start;
      gap: 8px;
    }
    
    .images-fashion-mnist {
      width: 32px;
      height: 32px;
      background-repeat: no-repeat;
      background-position: center;
    }
    """

    images = traittypes.Array(default_value=[]).tag(sync=True)

In [ ]:
import ipywidgets
import jscatter

images = ImagesWidget()

scatter = jscatter.Scatter(
    data=fashion_mnist,
    x='umapX',
    y='umapY',
    color_by='class',
    background_color='black',
)

ipywidgets.link((scatter.widget, 'selection'), (images, 'images'))

ipywidgets.AppLayout(center=scatter.show(), right_sidebar=images)

In [ ]:
config = dict(
    data=fashion_mnist,
    background_color='#111111',
    color_by='class',
    legend=True,
    axes=False,
    zoom_on_selection=True,
)

pca = jscatter.Scatter(x='pcaX', y='pcaY', **config)
tsne = jscatter.Scatter(x='tsneX', y='tsneY', **config)
umap = jscatter.Scatter(x='umapX', y='umapY', **config)
cae = jscatter.Scatter(x='caeX', y='caeY', **config)

jscatter.compose(
    [(pca, "PCA"), (tsne, "t-SNE"), (umap, "UMAP"), (cae, "CAE")],
    sync_selection=True,
    sync_hover=True,
    rows=2,
)